In [1]:
import pandas as pd
import numpy as np
import sys, os

In [16]:
#Pre-process the dataset
input_path='/fastscratch/c-panz/2021-09-14'
output_path='/pod/2/li-lab/Ziwei/Nanopore/daily/2021-09-14'
df=pd.read_csv(os.path.join(input_path, 'total.test.bed'), sep='\t')
df

,chr,start,read_id,strand,5hmC_prob,5mC_prob,5C_prob,label
0,KJ477685.1,204,3e547b1a-0a3e-4475-9b70-0cbd7c8c82aa,-,0.435397,0.085058,0.479545,2
1,KJ477685.1,261,3e547b1a-0a3e-4475-9b70-0cbd7c8c82aa,-,0.342189,0.500821,0.156990,2
2,KJ477685.1,261,fe8be0cc-f4c4-49f1-80db-c840a6c7dfe9,-,0.577034,0.057676,0.365290,2
3,KJ477685.1,266,3e547b1a-0a3e-4475-9b70-0cbd7c8c82aa,-,0.065316,0.861569,0.073115,2
4,KJ477685.1,266,fe8be0cc-f4c4-49f1-80db-c840a6c7dfe9,-,0.794235,0.028885,0.176880,2
...,...,...,...,...,...,...,...,...
592,J02459.1,60,24f224d8-1dda-4fbb-90a1-d299ca9b3273,-,0.025089,0.006866,0.968045,0
593,J02459.1,60,250b5169-f75c-492f-91ee-6a6ce8409ed9,-,0.051862,0.004506,0.943631,0
594,J02459.1,60,2539c0b5-58ba-4133-9571-176c5334c386,-,0.032449,0.009848,0.957703,0
595,J02459.1,60,25634907-a2ae-4bf9-967d-4d18b7c384b3,-,0.080185,0.002185,0.917630,0


In [17]:
#Splitting the data into independent and dependent variables
df_feature = df.loc[:,['5hmC_prob','5mC_prob','5C_prob']].values
df_class = df.loc[:,['label']].values
df_class = np.squeeze(df_class) #Convert the label into 1d-array

In [22]:
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_validate
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier
from statistics import mean

#Randomly spilt dataset to traing/testing dataset with the original ratio
X_train, X_test, y_train, y_test = train_test_split(df_feature, 
                                                    df_class, 
                                                    test_size=0.2, 
                                                    stratify=df_class)

# Hyperparameter tuning with Grid Search

Most important hyperparameters of Random Forest:

n_estimators = n of trees

max_features = max number of features considered for splitting a node

max_depth = max number of levels in each decision tree

min_samples_split = min number of data points placed in a node before the node is split

min_samples_leaf = min number of data points allowed in a leaf node

bootstrap = method for sampling data points (with or without replacement)

In [44]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

rf_model = RandomForestClassifier(random_state=42)
#Create Stratified K-fold cross validation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
#https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
# 5 * 2 * 4 * 3 * 3 = 360

param_grid = {
    'n_estimators'     : [10, 50, 100, 200, 300],
#    'max_features'     : ['sqrt', None],
#    'max_depth'        : [5, 10, 25, 50],
#    'min_samples_split': [5, 8, 10], #The minimum sample number to split an internal node
#    'min_samples_leaf' : [3, 4, 5]
#    'bootstrap'        : [True, False]
}


#scores = ['precision', 'recall']
#https://medium.com/sfu-cspmp/surviving-in-a-random-forest-with-imbalanced-datasets-b98b963d52eb
scores = ['accuracy', 'recall_macro', 'f1_macro']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    ##Grid search
    clf = GridSearchCV(estimator = rf_model, param_grid = param_grid, scoring= score, cv=cv, 
                       verbose=0, return_train_score=True, n_jobs=-1)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()


# Tuning hyper-parameters for accuracy

Best parameters set found on development set:

{'n_estimators': 300}

Grid scores on development set:

0.936 (+/-0.083) for {'n_estimators': 10}
0.936 (+/-0.081) for {'n_estimators': 50}
0.937 (+/-0.082) for {'n_estimators': 100}
0.937 (+/-0.084) for {'n_estimators': 200}
0.938 (+/-0.085) for {'n_estimators': 300}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

              precision    recall  f1-score   support

           0       0.90      0.93      0.91        40
           1       0.95      0.98      0.97        60
           2       0.82      0.70      0.76        20

    accuracy                           0.92       120
   macro avg       0.89      0.87      0.88       120
weighted avg       0.91      0.92      0.91       120


# Tuning hyper-parameters for recall_macro

Best parameters set found on development set:

{'n_estimators': 200}

Grid scores

In [1]:
#https://www.kaggle.com/emanueleamcappella/random-forest-hyperparameters-tuning

from sklearn.model_selection import GridSearchCV

rf_model = RandomForestClassifier(random_state=42)

#https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
# 5 * 2 * 4 * 3 * 3 = 360

param_grid = {
    'n_estimators'     : [10, 50, 100, 200, 300],
#    'max_features'     : ['sqrt', None],
#    'max_depth'        : [5, 10, 25, 50],
#    'min_samples_split': [5, 8, 10], #The minimum sample number to split an internal node
#    'min_samples_leaf' : [3, 4, 5]
#    'bootstrap'        : [True, False]
}

#Create Stratified K-fold cross validation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

##Grid search
grid_rf = GridSearchCV(estimator = rf_model, param_grid = param_grid, scoring='f1_macro', cv=cv, 
                       verbose=0, return_train_score=True, n_jobs=-1)

grid_rf.fit(X_train, y_train)

print("Best parameters set found on training set:\n")
print(grid_rf.best_params_)

print("Bestf1_score on validation set in CV:\n")
print(grid_rf.best_score_)

NameError: name 'RandomForestClassifier' is not defined